In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import numpy as np
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

%matplotlib inline

In [2]:
data = pd.read_csv('train.csv')

In [3]:
df1 = data.copy()

In [4]:
df1.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [5]:
df1.loc[(df1['LifeSquare']-df1['Square'])>10, 'LifeSquare'] = df1.loc[(df1['LifeSquare']-df1['Square'])>10, 'LifeSquare'] * 0.1
df1.loc[(df1['Square'] - df1['LifeSquare'])< -10, 'LifeSquare'] = df1.loc[(df1['Square'] - df1['LifeSquare'])< -10, 'LifeSquare'] * 0.1
df1.loc[df1['Square']>400, 'Square'] = df1.loc[df1['Square']>400, 'Square'] * 0.1

In [6]:
def prepare_square(df):
    df.loc[df['Square'] < 20, 'Square'] = 20
    df.loc[df['Square']<df['LifeSquare'], 'Square'] = df['LifeSquare']
    return df

In [7]:
df1.loc[df1['HouseYear']==4968, 'HouseYear'] = 1968

In [8]:
df1.loc[df1['HouseYear']==20052011, 'HouseYear'] = 2005

In [9]:
def prepare_floor(df):
    df.loc[df['HouseFloor']==0, 'HouseFloor'] = df.loc[df['HouseFloor']==0, 'Floor']
    df.loc[df['HouseFloor']<df['Floor'], 'HouseFloor'] = df.loc[df['HouseFloor']<df['Floor'], 'Floor']
    return df

In [10]:
def prepare_rooms(df):
    df.loc[df['Rooms']>11, 'Rooms'] = 1
    df.loc[df['Rooms']==0, 'Rooms'] = 2
    df.loc[df['Rooms']==10, 'Rooms'] = 2
    return df

In [11]:
df1['Rooms'].value_counts()

2.0     3880
1.0     3705
3.0     2235
4.0      150
5.0       18
0.0        8
10.0       2
19.0       1
6.0        1
Name: Rooms, dtype: int64

In [12]:
def prepare_lifesquare(df):
    df.loc[df['LifeSquare']>300, 'LifeSquare'] = df.loc[df['LifeSquare']>400, 'LifeSquare'] * 0.1
    df.loc[df['LifeSquare']<5, 'LifeSquare'] = df.loc[df['LifeSquare']<5, 'LifeSquare'] * 10
    df.loc[df['LifeSquare']<5, 'LifeSquare'] = df.loc[df['LifeSquare']<5, 'LifeSquare'] * 10
    return df

In [13]:
def prepare_kitchen(df):
    df.loc[df['KitchenSquare']>50, 'KitchenSquare'] = 8
    df.loc[df['KitchenSquare']>df['Square'], 'KitchenSquare'] = df.loc[df['KitchenSquare']>df['Square'], 'KitchenSquare']*0.1
    df.loc[df['KitchenSquare']==1, 'KitchenSquare'] = 4
    return df

In [14]:
df1.loc[df1['KitchenSquare']==2014, 'KitchenSquare'] = 2

In [15]:
df1.loc[df1['KitchenSquare']==1970, 'KitchenSquare'] = 5

In [16]:
df1.drop('Healthcare_1', axis=1, inplace=True)

In [17]:
def fillna_lifesquare(df, df_source):
    df['LifeSquare']=df['LifeSquare'].fillna(df_source['Square']-10)
    return df

In [18]:
df1 = prepare_lifesquare(df1)
df1 = prepare_square(df1)
df1 = prepare_floor(df1)
df1 = prepare_rooms(df1)
df1 = prepare_kitchen(df1)
df1 = fillna_lifesquare(df1, df1)

In [19]:
df1 = pd.get_dummies(df1)

In [20]:
def join_stats1(df,stats1,source_df,mode='train'):
    df=pd.merge(df,stats1,on=['DistrictId','Rooms'],how='left')
    if mode=='test':
        df['mean_price']=df['mean_price'].fillna(source_df['mean_price'].mean())
    return df

In [21]:
stats1=df1.groupby(['DistrictId','Rooms'],as_index=False)[['Price']].mean().rename(columns={'Price':'mean_price'})

In [22]:
df1=join_stats1(df1,stats1,df1)

In [23]:
features=['DistrictId', 'Rooms','Square','LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'mean_price']

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train,X_test,y_train,y_test=train_test_split(df1.loc[:,features],df1['Price'], test_size=0.35,random_state=42)

In [26]:
from sklearn.linear_model import LinearRegression as LR, Ridge
from sklearn.ensemble import RandomForestRegressor as RF

In [27]:
model1 = LR()
model1.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [28]:
y_pred1 = model1.predict(X_test)

In [29]:
from sklearn.metrics import mean_squared_error as mse, r2_score as r2

In [30]:
print(mse(y_test,y_pred1), r2(y_test,y_pred1))

2525410423.331776 0.7074419757953949


In [31]:
model2 = Ridge(random_state=42, alpha=0.05)
model2.fit(X_train,y_train)

Ridge(alpha=0.05, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=42, solver='auto', tol=0.001)

In [32]:
y_pred2 = model2.predict(X_test)
print(mse(y_test,y_pred2), r2(y_test,y_pred2))

2525409025.5491133 0.7074421377225536


In [33]:
clf = RF(random_state=42)

In [34]:
parameters = {
    'n_estimators': [1000, 1200],
    'min_samples_leaf':[2, 3],
    'max_depth': [10,15],
}

model3 = GridSearchCV(
    estimator=clf,
    param_grid=parameters,
    cv=2,
)

In [35]:
model3.fit(X_train,y_train)

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [1000, 1200], 'min_samples_leaf': [2, 3], 'max_depth': [10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [36]:
model3.best_params_

{'max_depth': 15, 'min_samples_leaf': 3, 'n_estimators': 1200}

In [37]:
clf = model3.best_estimator_
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1200, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [38]:
y_pred3 = clf.predict(X_test)
print(mse(y_test,y_pred3), r2(y_test,y_pred3))
#2115257504.0661216 0.7549564417901157

2115257504.0661216 0.7549564417901157


In [40]:
test = pd.read_csv('test.csv')

In [41]:
tst = test.copy()

In [42]:
tst.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,2623.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1146.657263,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1044.744231,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,325.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,1548.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


In [43]:
tst['Rooms'].value_counts()

2.0     2030
1.0     1769
3.0     1099
4.0       90
5.0        7
0.0        2
6.0        2
17.0       1
Name: Rooms, dtype: int64

In [44]:
tst.drop('Healthcare_1', axis=1, inplace=True)

In [45]:
tst = prepare_lifesquare(tst)
tst = prepare_square(tst)
tst = prepare_floor(tst)
tst = prepare_rooms(tst)
tst = prepare_kitchen(tst)
tst = fillna_lifesquare(tst, tst)

In [46]:
tst = pd.get_dummies(tst)

In [47]:
tst=join_stats1(tst,stats1,df1,mode='test')

In [49]:
y_result=pd.Series(clf.predict(tst.loc[:,features]))

In [52]:
result = pd.concat([tst['Id'], y_result], axis=1).rename(columns = {0:'Price'})

In [53]:
result.to_csv('DDubovskaia_predictions.csv',index=None)